In [7]:
pathneg= "./aclImdb/train/neg/"
pathpos="./aclImdb/train/pos/"

In [2]:
import glob, os


In [9]:
negfiles=[]
posfiles=[]
for file in glob.glob(pathneg+"*.txt"):
    f = open(file, "r")
    negfiles.append(f.read())
    negfiles=[]
for file in glob.glob(pathpos+"*.txt"):
    f = open(file, "r")
    posfiles.append(f.read())

In [11]:
neg=list(zip(negfiles,[0]*len(negfiles)))
pos=list(zip(posfiles,[1]*len(posfiles)))

In [13]:
sentiments = neg + pos

In [1]:
import glob, os
def loadimdb_train(train=True):
    if(train == True):
        pathneg= "./aclImdb/train/neg/"
        pathpos="./aclImdb/train/pos/"
    else:
        pathneg= "./aclImdb/test/neg/"
        pathpos="./aclImdb/test/pos/"
    negfiles=[]
    posfiles=[]
    for file in glob.glob(pathneg+"*.txt"):
        f = open(file, "r")
        negfiles.append(f.read())
        negfiles=[]
    for file in glob.glob(pathpos+"*.txt"):
        f = open(file, "r")
        posfiles.append(f.read())
        
    neg=list(zip(negfiles,[0]*len(negfiles)))
    pos=list(zip(posfiles,[1]*len(posfiles)))
    sentiments = neg + pos
    return(sentiments)
